In [1]:
!pip install -Uq unsloth transformers trl vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.

In [1]:
# importing necessary libraries
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset, Dataset
import re
from trl import GRPOConfig, GRPOTrainer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-17 03:28:55 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-17 03:28:56 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = 1024,
    load_in_4bit = True,
    fast_inference = True
)

==((====))==  Unsloth 2025.6.2: Fast Qwen2 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 4.88 GB. Also swap space = 0 GB.
WARNING 06-17 03:29:20 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 06-17 03:29:47 [config.py:717] This model supports multiple tasks: {'score', 'generate', 'embed', 'c

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-17 03:29:56 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 06-17 03:29:56 [model_runner.py:1140] Model loading took 2.4392 GiB and 5.252960 seconds
INFO 06-17 03:30:00 [worker.py:287] Memory profiling takes 3.33 seconds
INFO 06-17 03:30:00 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.50) = 7.30GiB
INFO 06-17 03:30:00 [worker.py:287] model weights take 2.44GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 3.79GiB.
INFO 06-17 03:30:01 [executor_base.py:112] # cuda blocks: 6894, # CPU blocks: 0
INFO 06-17 03:30:01 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 107.72x
INFO 06-17 03:30:01 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out

Capturing CUDA graph shapes:   0%|          | 0/27 [00:00<?, ?it/s]

INFO 06-17 03:30:43 [model_runner.py:1592] Graph capturing finished in 43 secs, took 0.56 GiB
INFO 06-17 03:30:43 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 47.38 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm', 'q_norm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm', 'q_norm']


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

Unsloth 2025.6.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()


def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split]
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })
    return data

dataset = get_gsm8k_questions()

In [15]:
def correctness_reward_func(prompts, completions, answer, **kwargs):
    """
    Returns 1.0 for each generation if it exactly matches the gold answer, else 0.0.
    """
    flat_rewards = []
    # completions is List[batch_idx][gen_idx]['content']
    for i, gen_list in enumerate(completions):
        gold = answer[i]
        for gen in gen_list:
            resp = extract_xml_answer(gen['content'])
            flat_rewards.append(1.0 if resp == gold else 0.0)
    return flat_rewards  # length == batch_size * num_generations


def strict_format_reward_func(prompts, completions, answer=None, **kwargs):
    """
    Returns 0.5 for each generation that strictly follows the XML format, else 0.0.
    """
    pattern = re.compile(r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$", re.DOTALL)
    flat_rewards = []
    for gen_list in completions:
        for gen in gen_list:
            text = gen['content']
            flat_rewards.append(0.5 if pattern.match(text) else 0.0)
    return flat_rewards  # same length as above


In [16]:
training_args = GRPOConfig(
    use_vllm = True,
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 4,
    max_prompt_length = 300,
    max_completion_length = 650,
    max_steps = 250,
    save_steps = 100,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [17]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        correctness_reward_func,
        strict_format_reward_func
    ],
    args = training_args,
    train_dataset = dataset,
)

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 29,933,568/3,000,000,000 (1.00% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std,rewards / strict_format_reward_func / mean,rewards / strict_format_reward_func / std
1,0.000000,0.000000,0.000000,329.500000,323.000000,335.000000,0.000000,329.500000,323.000000,335.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,552.250000,451.000000,650.000000,0.250000,519.666687,451.000000,620.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,349.500000,294.000000,405.000000,0.000000,349.500000,294.000000,405.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.000000,0.250000,0.500000,324.250000,266.000000,367.000000,0.000000,324.250000,266.000000,367.000000,0.000000,0.250000,0.500000,0.000000,0.000000
5,0.000000,0.000000,0.000000,219.250000,192.000000,246.000000,0.000000,219.250000,192.000000,246.000000,0.000006,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,406.000000,217.000000,577.000000,0.000000,406.000000,217.000000,577.000000,0.000005,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,363.250000,247.000000,474.000000,0.000000,363.250000,247.000000,474.000000,0.000007,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,310.500000,230.000000,468.000000,0.000000,310.500000,230.000000,468.000000,0.000013,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,233.500000,203.000000,290.000000,0.000000,233.500000,203.000000,290.000000,0.000006,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,298.000000,253.000000,336.000000,0.000000,298.000000,253.000000,336.000000,0.000007,0.000000,0.000000,0.000000,0.000000
